In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

# Load the training and test data
train_data = pd.read_csv("kaggle_train.csv")
test_data = pd.read_csv("kaggle_test.csv")

# Extract the 'ID' column from the test data before scaling
test_ids = test_data['ID']
test_data = test_data.drop("ID", axis=1)

# Separate features and labels
X_train = train_data.drop("Labels", axis=1)
y_train = train_data["Labels"]

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
test_data = scaler.transform(test_data)

# Feature selection with RFE
num_features_to_select = 10  # Adjust this value as needed
rfe = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=num_features_to_select)
X_train = rfe.fit_transform(X_train, y_train)
test_data = rfe.transform(test_data)

# Split the training data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=50)

# Train a machine learning model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = model.predict_proba(X_val)[:, 1]

# Calculate the AUC-ROC score on the validation set
auc_score = roc_auc_score(y_val, y_val_pred)
print(f"Validation AUC-ROC Score: {auc_score}")

# Make predictions on the test set
test_predictions = model.predict_proba(test_data)[:, 1]

# Create a submission DataFrame
submission = pd.DataFrame({"ID": test_ids, "Labels": test_predictions})

# Save the submission to a CSV file
submission.to_csv("output.csv", index=False)

Validation AUC-ROC Score: 0.7645161290322581
